# Predicting Iris Species

## 1. Business understanding

In this notebook, we are going to build a predictive model learning to explore the capacity of machine learning to address the taxonomic impediment.

[Presentation slides](https://docs.google.com/presentation/d/1St9pn2o1xR4_S9GS4uoGl9S4r0JCo8nPppBFBWdf3DA/edit?usp=sharing)

## 2. Setup

### Coding constants

### Common imports

## Load the data

## 3. Data understanding

### Attributes variables distribution

### Target variable distribution

### Visualizing relationships between the target variable and attributes 

## 4. Data Processing

The data has been preprocessed already, so there is nothing to do here.

## 5. Data Modeling

### Assessing feature importance